In [ ]:
%load_ext ipy_pdcache

In [2]:
import os
import time
from io import StringIO

import numpy as np
import pandas as pd

import networkx as nx

#import onto2nx
import obonet
import pybiomart

import requests
import zstandard as zstd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

from gene_map import GeneMapper

# Parameters

In [3]:
disgenet_fname = '../../results/input/disgenet.tsv'
gwascatalog_fname = '../../results/input/gwas_catalog.tsv'
efo_fname = '../../results/input/efo.obo'
so_fname = '../../results/input/so.obo'

db_out_fname = '../../results/databases/initial.csv'
raw_veps_fname = '../../results/databases/vep.csv'

gwas_gene_source = 'mapped'
annotation_sources = {'hg19': '../../resources/vep_cache_hg19.csv.zst', 'hg38': '../../resources/vep_cache_hg38.csv.zst'}
#snp_filters = snakemake.config['snp_filters']

# Load DisGeNET

In [4]:
df_disgenet = pd.read_table(
    disgenet_fname, usecols=['snpId', 'diseaseId', 'diseaseName', 'source']
)

df_disgenet['snp_source'] = 'disgenet'
df_disgenet['diseaseIdType'] = 'UMLS_CUI'

In [5]:
df_disgenet.head()

,snpId,diseaseId,diseaseName,source,snp_source,diseaseIdType
0,rs1000005,C0200638,Eosinophil count procedure,GWASCAT,disgenet,UMLS_CUI
1,rs10000770,C0023467,"Leukemia, Myelocytic, Acute",GWASCAT,disgenet,UMLS_CUI
2,rs1000091588,C1858517,SPINAL MUSCULAR ATROPHY WITH RESPIRATORY DISTR...,CLINVAR,disgenet,UMLS_CUI
3,rs1000096,C1305855,Body mass index,GWASCAT,disgenet,UMLS_CUI
4,rs10001106,C0003868,"Arthritis, Gouty",GWASDB,disgenet,UMLS_CUI


# Load GWAS catalog

## Parse input

In [6]:
df_gwascat = pd.read_table(gwascatalog_fname, low_memory=False)

# df_gwascat = df_gwascat[['SNP_ID_CURRENT', 'MAPPED_TRAIT_URI', 'MAPPED_TRAIT']]
df_gwascat.dropna(
    subset=['SNP_ID_CURRENT', 'MAPPED_TRAIT_URI', 'MAPPED_TRAIT'], inplace=True
)
df_gwascat.rename(
    columns={
        'SNP_ID_CURRENT': 'snpId',
        'MAPPED_TRAIT_URI': 'diseaseId',
        'MAPPED_TRAIT': 'diseaseName',
    },
    inplace=True,
)

df_gwascat['snpId'] = df_gwascat['snpId'].apply(lambda x: f'rs{x}')
df_gwascat['snp_source'] = 'gwas_catalog'

df_gwascat['diseaseId'] = df_gwascat['diseaseId'].str.split(',')
df_gwascat = df_gwascat.explode('diseaseId')

df_gwascat['diseaseId'] = df_gwascat['diseaseId'].apply(lambda x: x.split('/')[-1])
df_gwascat['diseaseIdType'] = df_gwascat['diseaseId'].apply(lambda x: x.split('_')[0])

# convert BETA to odds ratio
df_gwascat['odds_ratio'] = df_gwascat['OR or BETA'].apply(
    lambda x: np.exp(x) if x < 1 else x
)

df_gwascat.head(1)

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,diseaseName,diseaseId,STUDY ACCESSION,GENOTYPING TECHNOLOGY,snp_source,diseaseIdType,odds_ratio
0,2011-07-20,21658281,Aouizerat BE,2011-06-10,BMC Cardiovasc Disord,www.ncbi.nlm.nih.gov/pubmed/21658281,GWAS for discovery and replication of genetic ...,Sudden cardiac arrest,"88 European ancestry cases, 517 European ances...",NaN,...,[1.06-1.16],Affymetrix [319222],N,sudden cardiac arrest,EFO_0004278,GCST001102,Genome-wide genotyping array,gwas_catalog,EFO,1.11


## Infer associated gene(s)

Possible columns:
* REPORTED GENE(S): gene reported by author
* MAPPED GENE: Gene(s) mapped to the strongest SNP (if SNP is intergenic uses upstream and downstream genes)
* SNP_GENE_IDS: Entrez Gene ID

In [7]:
df_gwascat[['REPORTED GENE(S)', 'MAPPED_GENE', 'SNP_GENE_IDS']].head()

,REPORTED GENE(S),MAPPED_GENE,SNP_GENE_IDS
0,intergenic,FTH1P5 - AL158050.2,NaN
1,intergenic,LINC02376 - AC073913.1,NaN
2,CHRNB4,CHRNB4,ENSG00000117971
3,intergenic,"MARCHF10, AC005821.1","ENSG00000173838, ENSG00000265702"
4,CDH4,"CDH4, CDH4","ENSG00000179242, ENSG00000280641"


In [8]:
if gwas_gene_source == 'reported':
    # are gene names, must be mapped to ENTREZ
    raw_genes = df_gwascat['REPORTED GENE(S)'].str.split(', ').tolist()

    gene_blacklist = {'intergenic', 'NR'}
    cur_genes = [
        g
        for gs in raw_genes
        if not isinstance(gs, float)
        for g in gs
        if g not in gene_blacklist
    ]  # isinstance(gs,float) -> gs==np.nan

    gm = GeneMapper()
    df_map = gm.query(
        id_list=cur_genes, source_id_type='Gene_Name', target_id_type='GeneID'
    )
    name2id = df_map.set_index('ID_from').to_dict()['ID_to']

    entrez_genes = [
        None if isinstance(gs, float) else [name2id[g] for g in gs if g in name2id]
        for gs in raw_genes
    ]
elif gwas_gene_source == 'mapped':
    # are already ENTREZ IDs
    raw_genes = df_gwascat['SNP_GENE_IDS'].str.split(', ').tolist()
    entrez_genes = [None if isinstance(gs, float) else gs for gs in raw_genes]
else:
    raise RuntimeError(f'Invalid gene source: "{gwas_gene_source}"')

In [9]:
df_gwascat['associated_genes'] = [
    None if gs is None else ','.join(gs) for gs in entrez_genes
]
df_gwascat[
    ['REPORTED GENE(S)', 'MAPPED_GENE', 'SNP_GENE_IDS', 'associated_genes']
].head()

,REPORTED GENE(S),MAPPED_GENE,SNP_GENE_IDS,associated_genes
0,intergenic,FTH1P5 - AL158050.2,NaN,None
1,intergenic,LINC02376 - AC073913.1,NaN,None
2,CHRNB4,CHRNB4,ENSG00000117971,ENSG00000117971
3,intergenic,"MARCHF10, AC005821.1","ENSG00000173838, ENSG00000265702","ENSG00000173838,ENSG00000265702"
4,CDH4,"CDH4, CDH4","ENSG00000179242, ENSG00000280641","ENSG00000179242,ENSG00000280641"


## Select relevant columns

In [10]:
df_gwascat.columns

Index(['DATE ADDED TO CATALOG', 'PUBMEDID', 'FIRST AUTHOR', 'DATE', 'JOURNAL',
       'LINK', 'STUDY', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE',
       'REPLICATION SAMPLE SIZE', 'REGION', 'CHR_ID', 'CHR_POS',
       'REPORTED GENE(S)', 'MAPPED_GENE', 'UPSTREAM_GENE_ID',
       'DOWNSTREAM_GENE_ID', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE',
       'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',
       'MERGED', 'snpId', 'CONTEXT', 'INTERGENIC', 'RISK ALLELE FREQUENCY',
       'P-VALUE', 'PVALUE_MLOG', 'P-VALUE (TEXT)', 'OR or BETA',
       '95% CI (TEXT)', 'PLATFORM [SNPS PASSING QC]', 'CNV', 'diseaseName',
       'diseaseId', 'STUDY ACCESSION', 'GENOTYPING TECHNOLOGY', 'snp_source',
       'diseaseIdType', 'odds_ratio', 'associated_genes'],
      dtype='object')

In [11]:
df_gwascat = df_gwascat[
    [
        'diseaseId',
        'snpId',
        'snp_source',
        'diseaseIdType',
        'odds_ratio',
        'associated_genes',
    ]
]
df_gwascat.head()

,diseaseId,snpId,snp_source,diseaseIdType,odds_ratio,associated_genes
0,EFO_0004278,rs190759,gwas_catalog,EFO,1.11,None
1,EFO_0004278,rs1823172,gwas_catalog,EFO,1.17,None
2,EFO_0004278,rs950776,gwas_catalog,EFO,1.09,ENSG00000117971
3,EFO_0004278,rs2251393,gwas_catalog,EFO,1.13,"ENSG00000173838,ENSG00000265702"
4,EFO_0004278,rs944260,gwas_catalog,EFO,1.10,"ENSG00000179242,ENSG00000280641"


# Combine sources

In [12]:
df = pd.concat([df_gwascat])  # df_disgenet,
df.head()

,diseaseId,snpId,snp_source,diseaseIdType,odds_ratio,associated_genes
0,EFO_0004278,rs190759,gwas_catalog,EFO,1.11,None
1,EFO_0004278,rs1823172,gwas_catalog,EFO,1.17,None
2,EFO_0004278,rs950776,gwas_catalog,EFO,1.09,ENSG00000117971
3,EFO_0004278,rs2251393,gwas_catalog,EFO,1.13,"ENSG00000173838,ENSG00000265702"
4,EFO_0004278,rs944260,gwas_catalog,EFO,1.10,"ENSG00000179242,ENSG00000280641"


# Load required ontologies

In [13]:
%%time

efo_graph = obonet.read_obo(efo_fname)
efo_graph.name = 'efo'
print(f"Graph name: {efo_graph.name}")
print(f"Number of nodes: {efo_graph.number_of_nodes()}")
print(f"Number of edges: {efo_graph.number_of_edges()}")

Graph name: efo
Number of nodes: 56615
Number of edges: 93111
CPU times: user 11.1 s, sys: 223 ms, total: 11.3 s
Wall time: 11.3 s


In [14]:
%%time

so_graph = obonet.read_obo(so_fname)
so_graph.name = 'so'
print(f"Graph name: {so_graph.name}")
print(f"Number of nodes: {so_graph.number_of_nodes()}")
print(f"Number of edges: {so_graph.number_of_edges()}")

Graph name: so
Number of nodes: 2393
Number of edges: 3126
CPU times: user 224 ms, sys: 2.47 ms, total: 227 ms
Wall time: 227 ms


# Label diseases

In [16]:
efo_label_map = {idx.replace(':', '_'): data.get('name', 'Unknown') for idx, data in efo_graph.nodes(data=True)}

In [17]:
print(len(efo_label_map))

56615


In [18]:
df['diseaseLabel'] = df['diseaseId'].map(efo_label_map)

In [19]:
df.head()

,diseaseId,snpId,snp_source,diseaseIdType,odds_ratio,associated_genes,diseaseLabel
0,EFO_0004278,rs190759,gwas_catalog,EFO,1.11,None,sudden cardiac arrest
1,EFO_0004278,rs1823172,gwas_catalog,EFO,1.17,None,sudden cardiac arrest
2,EFO_0004278,rs950776,gwas_catalog,EFO,1.09,ENSG00000117971,sudden cardiac arrest
3,EFO_0004278,rs2251393,gwas_catalog,EFO,1.13,"ENSG00000173838,ENSG00000265702",sudden cardiac arrest
4,EFO_0004278,rs944260,gwas_catalog,EFO,1.10,"ENSG00000179242,ENSG00000280641",sudden cardiac arrest


# Cancer classification

In [20]:
nodes_all = list(efo_graph.nodes())

# find all disease-nodes
nodes_disease = list(nx.ancestors(efo_graph, 'EFO:0000408')) + [
    'EFO:0000408'
]  # disease subtree (vs traits, ...)

# find all cancer diseases
nodes_cancer = list(nx.ancestors(efo_graph, 'MONDO:0004992')) + [
    'MONDO:0004992'
]  # cancer subtree

# assert nodes_cancer <= nodes_disease
# assert nodes_disease <= nodes_all  # ???
print(
    f'#cancer/#disease/#all: {len(nodes_cancer)}/{len(nodes_disease)}/{len(nodes_all)}'
)

#cancer/#disease/#all: 2955/19250/56615


In [21]:
nodes_disease = [node.replace(':', '_') for node in nodes_disease]
nodes_cancer = [node.replace(':', '_') for node in nodes_cancer]
nodes_all = [node.replace(':', '_') for node in nodes_all]

In [22]:
if 'EFO_0005842	' in nodes_cancer:
    print('yes')

In [23]:
tmp = []
for disease in tqdm(df['diseaseId'].unique()):
    if disease in nodes_disease:
        tmp.append({'diseaseId': disease, 'is_cancer': disease in nodes_cancer})
    else:
        tmp.append({'diseaseId': disease, 'is_cancer': np.nan})

df_iscancer = pd.DataFrame(tmp)
df_iscancer.head(5)

  0%|          | 0/2845 [00:00<?, ?it/s]

,diseaseId,is_cancer
0,EFO_0004278,False
1,EFO_0005842,NaN
2,EFO_0000341,False
3,EFO_0004340,False
4,EFO_0004329,NaN


# SNP annotations

## Retrieve VEP annotations

Variant consequence ontology: http://www.sequenceontology.org/browser/current_release

Description of variant types: https://www.ensembl.org/info/genome/variation/prediction/predicted_data.html

Raw data: ftp://ftp.ensembl.org/pub/release-98/variation/vep/

In [24]:
snps = df['snpId'].unique().tolist()
print(f'Retrieving annotations for {len(snps)} SNPs')

Retrieving annotations for 136371 SNPs


In [25]:
%%time
%%pdcache df_anno_raw $raw_veps_fname

df_list = []
for genome_assembly, annotation_url in annotation_sources.items():
    if os.path.isfile(annotation_url):
        print('Using local SNP annotations')

        if annotation_url.endswith('.zst'):
            with open(annotation_url, 'rb') as fd:
                data = fd.read()

            dctx = zstd.ZstdDecompressor()
            decompressed = dctx.decompress(data)

            tmp = pd.read_csv(StringIO(decompressed.decode()), index_col=0)
        else:
            tmp = pd.read_csv(annotation_url, index_col=0)
    else:
        print('Retrieving SNP annotations from Ensembl')

        tmp = None
        while tmp is None:
            try:
                server = pybiomart.Server(host=annotation_url)
                dataset = server.marts['ENSEMBL_MART_SNP'].datasets['hsapiens_snp']

                tmp = dataset.query(
                    attributes=[
                        'refsnp_id',
                        'chr_name',
                        'chrom_start',
                        'consequence_type_tv',
                        'ensembl_transcript_stable_id',
                    ],
                    filters={'snp_filter': snps},
                    use_attr_names=True,
                )
            except requests.HTTPError:
                # retry if network error occurred
                print('Next try...')
                time.sleep(10)

    tmp['genome_assembly'] = genome_assembly
    df_list.append(tmp)

df_anno_raw = pd.concat(df_list, ignore_index=True)

UsageError: Cell magic `%%pdcache` not found.


In [26]:
df_anno_raw.head()

NameError: name 'df_anno_raw' is not defined

## Convert annotations to usable format

In [ ]:
df_anno = df_anno_raw.copy()

# processing preparations
df_anno['chr_name'] = df_anno['chr_name'].astype(str)

# remove haplotypes (e.g. CHR_HSCHR6_MHC_COX_CTG1)
df_anno = df_anno[~df_anno['chr_name'].str.contains('_')]

# mark empty consequence as 'intergenic' (NaN in dataframe shows up as intergenic in VEP web-interface)
df_anno.loc[
    df_anno['consequence_type_tv'].isna(), 'consequence_type_tv'
] = 'intergenic_variant'

# select most frequent annotations
tmp = []
for (snp, genome_assembly), group in tqdm(
    df_anno.groupby(['refsnp_id', 'genome_assembly'])
):
    vep_counts = group['consequence_type_tv'].value_counts()
    top_count = vep_counts.max()

    # deterministically choose "some" top value
    top_vep = sorted(vep_counts[vep_counts == top_count].index)[0]

    match = group[group['consequence_type_tv'] == top_vep].iloc[0]
    tmp.append(match)
df_anno = pd.DataFrame(tmp)

# set column names
df_anno.drop('ensembl_transcript_stable_id', axis=1, inplace=True)

df_anno.rename(
    columns={
        'refsnp_id': 'snpId',
        'consequence_type_tv': 'variant_type',
        'chr_name': 'chromosome',
        'chrom_start': 'position',
    },
    inplace=True,
)

In [ ]:
df_anno.head()

## Group variant types

### Read sequence ontology (SO)

In [27]:
exon_subgraph = list(nx.ancestors(so_graph, 'SO:0001791')) + ['SO:0001791']
intron_subgraph = list(nx.ancestors(so_graph, 'SO:0001627')) + ['SO:0001627']
intergenic_subgraph = list(nx.ancestors(so_graph, 'SO:0001628')) + ['SO:0001628']

In [29]:
exon_subgraph = [node.replace(':', '_') for node in exon_subgraph]
intron_subgraph = [node.replace(':', '_') for node in intron_subgraph]
intergenic_subgraph = [node.replace(':', '_') for node in intergenic_subgraph]

### Find ontology labels

In [30]:
so_label_map = {data.get('name', 'Unknown'): idx.replace(':', '_') for idx, data in so_graph.nodes(data=True)}

### Classify variants

In [31]:
def classify_vep(vep):
    special_cases = {
        'NMD_transcript_variant': 'exonic',
        'mature_miRNA_variant': 'exonic',
        'splice_region_variant': 'exonic',  # can be either exon or intron
        'non_coding_transcript_variant': 'intronic',
    }

    vep_id = so_label_map[vep]
    if vep_id in exon_subgraph:
        assert vep_id not in intron_subgraph and vep_id not in intergenic_subgraph, vep
        return 'exonic'
    elif vep_id in intron_subgraph:
        assert vep_id not in exon_subgraph and vep_id not in intergenic_subgraph, vep
        return 'intronic'
    elif vep_id in intergenic_subgraph:
        assert vep_id not in intron_subgraph and vep_id not in exon_subgraph, vep
        return 'intergenic'
    else:
        return special_cases.get(vep, 'ambiguous')

In [71]:
df_anno['variant_group'] = df_anno['variant_type'].apply(classify_vep)
df_anno['variant_group'].value_counts()

NameError: name 'df_anno' is not defined

In [ ]:
df_anno.head()

## Sanity checks

In [ ]:
# assert that all SNPs have been annotated (TODO: make this rigorous)
# assert set(df_anno['snpId'].tolist()) == set(snps), set(snps) - set(df_anno['snpId'].tolist())
assert df_anno is not None
assert df_anno.shape[0] > 0

In [ ]:
# assert that all variant types have been grouped
assert df_anno['variant_group'].isna().sum() == 0, (
    df_anno[df_anno.variant_group.isna()]
    .drop_duplicates('variant_type')['variant_type']
    .tolist()
)

In [ ]:
# assert that variant type groups are reasonable
# assert set(df_anno['variant_group']) <= {'exonic', 'intronic', 'intergenic', 'ambiguous'}, df_anno['variant_group'].unique().tolist()

In [ ]:
# statistics
print('#SNPs in database:', df['snpId'].nunique(), f'({len(snps)})')
print('#annotated SNPs:', df_anno['snpId'].nunique())
print('#intersection:', len(set(df['snpId'].tolist()) & set(df_anno['snpId'].tolist())))

## Transform dataset

In [ ]:
def dummy_agg(x):
    assert len(x) <= 1
    return x


df_anno_trans = pd.pivot_table(
    df_anno,
    values=['chromosome', 'position', 'variant_type', 'variant_group'],
    index=['snpId'],
    columns=['genome_assembly'],
    aggfunc=dummy_agg,
).reset_index()

df_anno_trans.columns = [
    '_'.join(col).rstrip('_') for col in df_anno_trans.columns.values
]

In [ ]:
df_anno_trans.head()

# Merge data sources

In [ ]:
# initial aggregation
df_final = df.copy()
df_final.shape

In [ ]:
# cancer-classification
df_final = df_final.merge(df_iscancer, on='diseaseId')
df_final.shape

In [ ]:
# SNP annotation
df_final = df_final.merge(df_anno_trans, how='left')
df_final.shape

In [ ]:
df_final.head()

# Apply filters

## General filters

In [ ]:
# only keep diseases (and not e.g. traits)
df_final.dropna(subset=['is_cancer'], inplace=True)

In [ ]:
df_final.shape

## Variant type filters (only add marker)

In [ ]:
for filter_name, filter_query in snp_filters.items():
    for genome_assembly in annotation_sources.keys():
        idx = f'filter_{filter_name}_{genome_assembly}'

        df_final[idx] = False
        if filter_query is None:
            df_final[idx] = True
        else:
            match = df_final.query(
                filter_query.format(genome_assembly=genome_assembly)
            ).index
            df_final.loc[match, idx] = True

# Save result

In [ ]:
df_final.head()

In [ ]:
df_final.to_csv(db_out_fname, index=False)